1. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk sorszámozott listát a szálláshelyek adatairól!&nbsp;</span> 

a. A lista legyen a szálláshely típusa szerint, azon belül hely szerint, majd a neve szerint növekvően rendezve! 

b. A számozás típusonként kezdődjön újra!

In [ ]:
SELECT ROW_NUMBER() 
       OVER(PARTITION BY TIPUS ORDER BY HELY, SZALLAS_NEV) AS 'Sorsz',
       *
FROM Szallashely
ORDER BY TIPUS, HELY, SZALLAS_NEV

2. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk lekérdezést, amely a foglalások adatait jeleníti meg!&nbsp;</span> 

a. A lista tartalmazzon két új oszlopot, amelyek az adott ügyfél legrövidebb, illetve leghosszabb foglalásának hosszát (a foglalt napok számát) mutatják meg! 

b. Az oszlopokat nevezzük el értelemszerűen!

In [ ]:
SELECT *,
       MIN(DATEDIFF(DAY, METTOL, MEDDIG)) 
	   OVER(PARTITION BY UGYFEL_FK) AS 'Legrövidebb foglalás',
	   MAX(DATEDIFF(DAY, METTOL, MEDDIG)) 
	   OVER(PARTITION BY UGYFEL_FK) AS 'Leghosszabb foglalás'
FROM Foglalas

```
3. Készítsük listát, amely megjeleníti az ügyfelek azonosítóját, nevét és a foglalásainak kezdő- és befejező dátumát, valamint azt, hogy a foglalás összesen hány főre történt! 
```

a. Egy új oszlopban jelenítsük meg azt is, hogy az ügyfél előző három foglalása összesen hány főre történt!

![](https://moodle.uni-corvinus.hu/theme/image.php/adaptable/atto_h5p/1709548862/icon)

In [ ]:
SELECT v.USERNEV, v.NEV, f.METTOL, f.MEDDIG,
       f.FELNOTT_SZAM + f.GYERMEK_SZAM AS 'Személyek száma',
	   SUM(f.FELNOTT_SZAM+f.GYERMEK_SZAM)
	   OVER(PARTITION BY f.UGYFEL_FK ORDER BY FOGLALAS_PK
	   ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) 
	   AS 'Személyek száma összesen az előző 3 foglalás alapján'
FROM Vendeg v JOIN Foglalas f ON f.UGYFEL_FK = v.USERNEV

4. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk listát arról, hogy az egyes szálláshelyeken évente hány foglalás történt!&nbsp;</span> 

a. A lista jelenítse meg a szálláshely azonosítóját, nevét, az évet, és a foglalások számát!

b. Egy új oszlopban jelenítsük meg a szálláshely eddigi éves foglalásainak számát is (az aktuálisat is beleértve). 

c. Az évnél a METTOL dátumot vegyük figyelembe! 

d. A lista ne tartalmazzon duplikált sorokat!

In [ ]:
SELECT DISTINCT sz.SZALLAS_ID, sz.SZALLAS_NEV, YEAR(f.METTOL) AS 'ÉV',
       COUNT(*) OVER(PARTITION BY sz.SZALLAS_ID, YEAR(f.METTOL)) 
	   AS 'Éves foglalásszám',
	   COUNT(*) OVER(PARTITION BY sz.SZALLAS_ID ORDER BY YEAR(f.METTOL) 
	   RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
	   AS 'Eddigi éves foglalásszám összesen'
FROM Foglalas f JOIN Szallashely sz ON f.SZOBA_FK = sz.SZALLAS_ID

5. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsük listát a szálláshelyekről! A lista tartalmazza a szálláshely azonosítóját, nevét, és a csillagok számát csillagszám szerint csökkenő, azon belül szállásnév szerint növekvő sorrendben!&nbsp;</span> 

a. Vegyünk fel két új oszlopot, amely a sorrend szerint előző szálláshely nevét, illetve azonosítóját is megjeleníti!

In [ ]:
SELECT  SZALLAS_ID, SZALLAS_NEV, CSILLAGOK_SZAMA, 
        LAG(szallas_nev) OVER (ORDER BY CSILLAGOK_SZAMA DESC, 
		szallas_nev) 
		AS 'Előző szállás neve',
		LAG(szallas_id) OVER (ORDER BY CSILLAGOK_SZAMA DESC, 
		szallas_nev) 
		AS 'Előző szállás azonosítója'		
FROM Szallashely 
--ORDER BY CSILLAGOK_SZAMA DESC, SZALLAS_NEV

6. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Listázzuk azon termékeket, amelyekből az átlagosnál kevesebb van raktáron!&nbsp;</span> 

a. Csak a termék kódja és megnevezése jelenjen meg!

In [ ]:
SELECT termekkod, megnevezes
FROM Termek
WHERE keszlet <
(
 SELECT AVG(keszlet)
 FROM Termek
)

7. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Listázzuk azon ügyfeleket, akik még nem fizettek bankkártyával!&nbsp;</span> 

a. Csak az ügyfelek neve jelenjen meg!

In [ ]:
SELECT u.nev
FROM Ugyfel u
WHERE NOT EXISTS
(
 SELECT *
 FROM Rendeles r JOIN Ugyfel u2 ON r.LOGIN = u2.LOGIN
 WHERE u.LOGIN = u2.LOGIN AND r.FIZ_MOD ='Bankkártya'
)

8. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Mennyibe kerül a második legnagyobb listaárú termék?</span>

In [ ]:
SELECT max(listaar)
FROM Termek 
WHERE listaar <
(
 SELECT max(LISTAAR)
 FROM Termek
)

9. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Melyik raktár(ak)ban van az a termék, amelyből a legnagyobb készlet van?&nbsp;</span> 

a. Csak a raktár(ak) neve jelenjen meg!

In [4]:
SELECT r.RAKTAR_NEV
FROM Raktar r JOIN Termek t ON r.RAKTAR_KOD = t.RAKTAR_KOD
WHERE t.keszlet =
(
  SELECT MAX(keszlet)
  FROM Termek
)

(2 rows affected)

Total execution time: 00:00:00.012

RAKTAR_NEV
Veresegyházi
Báthory


10. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Melyek azok a termékek, amelyek listaára nagyobb, mint bármelyik olyan terméké, amelynek nevében az Ajándék szó szerepel?&nbsp;</span> 

a. A termékek kódját és nevét is jelenítsük meg!

In [ ]:
SELECT MEGNEVEZES
FROM Termek 
WHERE LISTAAR > ALL
(
 SELECT LISTAAR
 FROM Termek
 WHERE MEGNEVEZES LIKE '%Ajándék%'
)

11. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Melyik hónapokban adtak le az ügyfelek több rendelést, mint januárban?&nbsp;</span> 

a. Elég a hónapok sorszámait megjeleníteni. 

b. A listát rendezzük a hónapok száma szerint növekvő sorrendbe!

In [5]:
SELECT MONTH(rend_datum)
FROM Rendeles
GROUP BY MONTH(rend_datum)
HAVING COUNT(*)>
(
 SELECT COUNT(*)
 FROM Rendeles
 WHERE MONTH(rend_datum)=1
)
ORDER BY 1

(5 rows affected)

Total execution time: 00:00:00.029

(No column name)
3
7
8
10
12


12. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Melyek azok a termékek, amelyek listaára az átlagostól max. 20%-kal tér el?&nbsp;</span> 

a. A termék kódját és nevét is jelenítsük meg!

In [6]:
SELECT termekkod, megnevezes
FROM Termek 
WHERE LISTAAR >=
(
 SELECT 0.8*AVG(listaar)
 FROM Termek
)

AND LISTAAR <=
(
 SELECT 1.2*AVG(listaar)
 FROM Termek
)

(26 rows affected)

Total execution time: 00:00:00.048

termekkod,megnevezes
01100094T,Fénymásoló papír DataCopy
01100095T,Fénymásoló papír GetCopy
01100096T,Fénymásoló papír Qconnect
01100505T,Fehér másoló A/4
02040194T,"Hibajav.roll.Pritt 4,2mm"
02060038T,Bélyegző IDEAL 50
02060039T,Bélyegző párna
02080454T,Lyukasztó BOXER
04020445T,Határidő napló 2010-s
06040186T,Gyűrűskönyv színes


13. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Készítsünk listát (sorszámonként) az egyes rendelések teljes összegéről!&nbsp;</span> 

a. Szűrjünk azon tételekre, ahol ez az érték nagyobb, mint Viktor Éva rendeléseinek teljes összege! 

b. A teljes összeg oszlopot nevezzük el értelemszerűen!

In [ ]:
SELECT r.SORSZAM, SUM(rt.MENNYISEG*rt.EGYSEGAR) AS 'Rendelés összege'
FROM Rendeles r JOIN Rendeles_tetel rt ON r.SORSZAM = rt.SORSZAM
GROUP BY r.SORSZAM
HAVING SUM(rt.MENNYISEG*rt.EGYSEGAR)>
(
 SELECT SUM(rt.MENNYISEG*rt.EGYSEGAR)
 FROM Rendeles r JOIN Rendeles_tetel rt ON r.SORSZAM = rt.SORSZAM
      JOIN Ugyfel u ON r.LOGIN = u.LOGIN
 WHERE u.NEV='Viktor Éva'
)


14. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Listázzuk azokat a termékkategóriákat, amelyből 2017 januárban nem történt rendelés!&nbsp;</span> 

a. Elég a kategóriák nevét megjeleníteni!

In [ ]:
SELECT tk.KAT_NEV
FROM Termekkategoria tk 
WHERE tk.KAT_ID NOT IN
(
 SELECT t.KAT_ID
 FROM Rendeles r JOIN Rendeles_tetel rt ON r.SORSZAM = rt.SORSZAM
      JOIN Termek t ON rt.TERMEKKOD = t.TERMEKKOD
 WHERE tk.KAT_ID = t.KAT_ID 
       AND r.REND_DATUM BETWEEN '2017.01.01' AND '2017.01.31'
)

15. <span style="background-color: rgb(255, 255, 255); color: rgb(0, 0, 0); font-family: &quot;Open Sans&quot;, sans-serif; font-size: 14.44px;">Listázzuk azon rendelési tételeket, amelynek értéke nagyobb, mint a legdrágább termék listaára azon termékek közül, amelyek szülő kategóriája a ruha!</span>

In [ ]:
SELECT rt.*
FROM Rendeles_tetel rt
WHERE rt.MENNYISEG * rt.EGYSEGAR >
(
  SELECT max(t.listaar)
  FROM Termek t
  WHERE t.KAT_ID IN
  (
    SELECT tk.kat_id
	FROM Termekkategoria tk
	     JOIN Termekkategoria tk2 ON tk.SZULO_KAT = tk2.KAT_ID
	WHERE tk2.KAT_NEV='ruha'
  )
)